In [ ]:
import requests
from lxml import etree
import json
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from random import randrange
import time
from datetime import datetime
import os, sys
import warnings
warnings.filterwarnings("ignore", message="use options instead of chrome_options")

In [ ]:
# https://medium.com/analytics-vidhya/effortlessly-automate-your-python-scripts-cd295697dff6
# where to save the csv file
_DATA_CDC_PATH_ = "/Users/xingruchen/Dropbox/COV/Vaccine/data/CDC/csv/"
_LOG_PATH_ = "/Users/xingruchen/Dropbox/COV/Vaccine/data/CDC/"

Doses distributed and people initiating vaccination (1st dose received) are for both Moderna and Pfizer BioNTech COVID-19 vaccine and reflect current data available as of 9:00am ET on the day of reporting. Data will be regularly updated on Monday, Wednesday and Friday. Updates will occur the following day when reporting coincides with a federal holiday.

Healthcare providers report doses to federal, state, territorial, and local agencies up to 72 hours after administration. There may be additional lag for data to be transmitted from the federal, state, territorial, or local agency to CDC. 

#### A large difference between the number of doses distributed and the number of people initiating vaccination is expected at this point in the COVID vaccination program due to several factors, including delays in reporting of administered doses and management of available vaccine stocks by jurisdictions and federal pharmacy partners.

Numbers reported on CDC’s website are validated through a submission process with each jurisdiction and may differ from numbers posted on other websites. Differences between reporting jurisdictions and CDC’s website may occur due to the timing of reporting and website updates. The process used for reporting doses distributed or people vaccinated displayed by other websites may differ.

When the “Rate per 100,000” metric is selected for both doses distributed and people initiating vaccination (1st dose received), federal entities will display as n/a because population-based rates are not applicable. Doses distributed and administered for federal entities will display when the “Counts” metric is selected.

#### Doses distributed are cumulative counts of COVID-19 vaccine doses recorded as shipped in the Centers for Disease Control and Prevention’s (CDC) Vaccine Tracking System (VTrckS) since December 13, 2020.

#### People initiating vaccination (1st dose received) are cumulative counts of individual COVID-19 vaccine first doses administered as reported to the CDC by state, territorial, and local public health agencies and four federal entities (Bureau of Prisons, Department of Defense, Indian Health Service, and Veterans Health Administration) since December 14, 2020. 
For information pertaining to Department of State’s COVID-19 vaccination program, please contact the Office of Press Operations at PAPressDuty@state.gov. Veterans Health Administration totals include employees, veteran patients, and other federal partners vaccinated by VHA.

#### Long-term care facility (LTCF) data is a subset of the overall national data, specific to the Federal Pharmacy Partnership for Long-term Care (LTC) Program, and primarily includes skilled nursing and assisted living facilities. Doses distributed refers to doses distributed to pharmacy partners to administer onsite at LTCFs, and people initiating vaccination (1st dose received) includes both LTCF residents and staff vaccinated through the program. 

This data does not include doses distributed and administered to LTCF residents and staff outside the Federal Pharmacy Partnership Program. Vaccine administration through the federal program launched nationally on December 21st for Pfizer vaccine and on December 28th for Moderna vaccine. As of December 29, 2020, a total of 52 jurisdictions have started the program. Program start dates vary based on the jurisdiction. A difference between the number of doses distributed to pharmacy partners and the number of people initiating vaccination is expected because jurisdictions are transferring doses to pharmacy partners in advance to prepare for vaccination clinics in subsequent weeks.

#### Rates per 100,000 population use the U.S. Census Bureau 2019 American Community Survey 1-year population estimates. 

2018 population estimates are used for American Samoa, Federated States of Micronesia, Guam, Northern Mariana Islands, Palau, Republic of Marshall Islands, and United States Virgin Islands.

In [ ]:
class CDCSpider(object):
    '''
    A CDC spider 
    '''
    def __init__(self):
        '''
        initialization
        :return: None
        '''
        self.url = 'https://covid.cdc.gov/covid-data-tracker/#vaccinations'
        # rate: per 100k
        self.headers = ['jurisdiction', 'distribution', 'initiate_1st', 
                        'distribution_rate', 'initiate_1st_rate', 'distribution_LTCF', 'initiation_1st_LTCF']
        self.df = pd.DataFrame(columns = self.headers)
    
    def start_driver(self):
        '''
        Open headless chromedriver
        '''
        options = webdriver.ChromeOptions()
        prefs = {"download.default_directory" : _DATA_CDC_PATH_}
        options.add_experimental_option("prefs", prefs)
        self.driver = webdriver.Chrome(executable_path = r'/usr/local/bin/chromedriver', chrome_options = options)
        self.driver.maximize_window()
        time.sleep(3)
        print('[start_driver] driver started')
        
    def close_driver(self):
        '''
        Close chromedriver
        '''
        self.driver.quit()
        print('[close_driver] driver closed')
        
    def get_page(self, url):
        '''
        Tell the browser to get a page
        :param url: webpage url
        '''
        self.driver.get(url)
        time.sleep(randrange(6))
        print('[get_page] url: {}'.format(url))
        time.sleep(randrange(3))
        
    def get_headers(self):
        '''
        Get column names of df
        :return: list of column names
        '''
        # Total Doses Distributed
        col_2 = self.driver.find_element_by_xpath('//*[@id="vaccinations-banner-wrapper"]/div[1]/div/div[1]/div/div/h4').text
        # Total Number of People Initiating Vaccination (1st Dose Received)
        col_3 = self.driver.find_element_by_xpath('//*[@id="vaccinations-banner-wrapper"]/div[1]/div/div[2]/div[1]/div/h4').text
        # Doses Distributed for Use in Long-Term Care Facilities
        col_6 = self.driver.find_element_by_xpath('//*[@id="vaccinations-banner-wrapper"]/div[2]/div/div[1]/div/div/h4').text
        # Number of People Initiating Vaccination (1st Dose Received) in Long-Term Care Facilities
        col_7 = self.driver.find_element_by_xpath('//*[@id="vaccinations-banner-wrapper"]/div[2]/div/div[2]/div[1]/div/h4').text
        
        cols = [col_2, col_3, col_6, col_7]
        for col in cols:
            print('[get_headers] headers: {0}'.format(col))
        
        time.sleep(randrange(3))
        
    
    def get_update_time(self):
        '''
        Get update time
        :return: date and time
        '''
        # CDC | Updated: Dec 30 2020 As of 9:00am ET
        update_text = self.driver.find_element_by_xpath('//*[@id="vaccinations-banner-wrapper"]/div[1]/div/div[2]/div[2]/small').text
        # Dec 30 2020
        update_date = update_text[update_text.find('Updated') + 9: update_text.find('Updated') + 20]
        # 9:00am
        update_time = update_text[update_text.find('As of') + 6: -3]
        # datetime.datetime(2020, 12, 30, 9, 0))
        update_time = datetime.strptime(update_date + ' ' + update_time, '%b %d %Y %I:%M%p')
        # 201230
        update_time_str = update_time.strftime("%y%m%d")
        
        self.update_time = update_time
        self.update_time_str = update_time_str
        time.sleep(randrange(3))
        
        print('[get_update_time] update time: {0}'.format(self.update_time))
        
    def get_df_total(self):
        '''
        Get the first row of df
        '''
        # Total Doses Distributed 
        # '12,409,050'
        distribution = self.driver.find_element_by_xpath('//*[@id="vaccinations-banner-wrapper"]/div[1]/div/div[1]/div/div/div').text
        # 12409050
        distribution = int(distribution.replace(',', ''))
        
        # Total Number of People Initiating Vaccination (1st Dose Received)
        # '2,794,588'
        initiate_1st = self.driver.find_element_by_xpath('//*[@id="vaccinations-banner-wrapper"]/div[1]/div/div[2]/div[1]/div/div').text
        # 2794588
        initiate_1st = int(initiate_1st.replace(',', ''))
        
        # Doses Distributed for Use in Long-Term Care Facilities
        # '2,166,200'
        distribution_LTCF = self.driver.find_element_by_xpath('//*[@id="vaccinations-banner-wrapper"]/div[2]/div/div[1]/div/div/div').text
        # 2166200
        distribution_LTCF = int(distribution_LTCF.replace(',', ''))
        
        # Number of People Initiating Vaccination (1st Dose Received) in Long-Term Care Facilities
        # '167,149'
        initiation_1st_LTCF = self.driver.find_element_by_xpath('//*[@id="vaccinations-banner-wrapper"]/div[2]/div/div[2]/div[1]/div/div').text
        # 167149
        initiation_1st_LTCF = int(initiation_1st_LTCF.replace(',', ''))
        
        self.df.loc[len(self.df)] = ['Total', distribution, initiate_1st, np.nan, np.nan, distribution_LTCF, initiation_1st_LTCF]
        
        self.df.to_csv(_DATA_CDC_PATH_ + 'vaccine_{0}.csv'.format(self.update_time_str), index = False)
        print('[get_df_total] distribution: {0}, initiate_1st: {1}'.format(distribution, initiate_1st))
        
        time.sleep(randrange(3))
        
    def get_df_state(self):
        '''
        Total Number of People Initiating Vaccination (1st Dose Received) Reported to the CDC by State/Territory 
        and for Selected Federal Entities per 100,000
        '''
        ################ States ################
        element = wait(self.driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "state")))
        elements = self.driver.find_elements_by_xpath('//*[@id="svg#vaccinations-map-wrapper"]/*[name()="g" and @class="state"]/*[name()="path"]')
        #elements = wait(self.driver, 6).until(EC.presence_of_element_located((By.XPATH, '//*[@id="svg#vaccinations-map-wrapper"]/*[name()="g" and @class="state"]/*[name()="path"]')))
        print('[get_df] number of elements: {0}'.format(len(elements)))
        for el in elements:
            hover = ActionChains(self.driver).move_to_element(el)
            hover.perform()
            time.sleep(2)
            tooltip = self.driver.find_element_by_css_selector('body > div.tooltip')
            tooltip = wait(self.driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body > div.tooltip')))
            text = tooltip.text.split('\n')
            if text == ['']:
                print('[get_df] empty A')
                continue
            jurisdiction = text[0]
            # 'Total Distributed: 103,375 (5,768 per 100k)'
            distribution = int(text[1][text[1].find(':') + 2 : text[1].find('(') - 1].replace(',', '')) # 103375
            distribution_rate = int(text[1][text[1].find('(') + 1 : text[1].find('per') - 1].replace(',', '')) # 5768
            # 'People Initiating Vaccination (1st Dose Received): 49,827 (2,780 per 100k)'
            initiate_1st = int(text[2].split(' ')[6].replace(',', '')) 
            initiate_1st_rate = int(text[2].split(' ')[7][1:].replace(',', ''))
            if jurisdiction not in self.df.jurisdiction.values:
                self.df.loc[len(self.df)] = [jurisdiction, distribution, initiate_1st, distribution_rate, initiate_1st_rate, 
                                             np.nan, np.nan]
                print('[get_df] {0} jurisdiction: {1}, distribution: {2}, initiate_1st: {3}'.format(len(self.df), jurisdiction, distribution, initiate_1st))
        
        while len(self.df) < 47: # some states are too random to reach b/c their locations
            self.driver.get(self.url)
            time.sleep(randrange(6))
            element = wait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "state")))
            elements = self.driver.find_elements_by_xpath('//*[@id="svg#vaccinations-map-wrapper"]/*[name()="g" and @class="state"]/*[name()="path"]')        
            for j in range(len(elements)):
                el = wait(self.driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#svg\#vaccinations-map-wrapper > g > path:nth-child({0})'.format(j + 1))))
                hover = ActionChains(self.driver).move_to_element(el)
                hover.perform()
                time.sleep(8)
                #tooltip = self.driver.find_element_by_css_selector('body > div.tooltip')
                tooltip = wait(self.driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body > div.tooltip')))
                text = tooltip.text.split('\n')
                if text == ['']:
                    print('[get_df] empty B')
                    continue
                jurisdiction = text[0]
                # 'Total Distributed: 103,375 (5,768 per 100k)'
                distribution = int(text[1][text[1].find(':') + 2 : text[1].find('(') - 1].replace(',', '')) # 103375
                distribution_rate = int(text[1][text[1].find('(') + 1 : text[1].find('per') - 1].replace(',', '')) # 5768
                # 'People Initiating Vaccination (1st Dose Received): 49,827 (2,780 per 100k)'
                initiate_1st = int(text[2].split(' ')[6].replace(',', '')) 
                initiate_1st_rate = int(text[2].split(' ')[7][1:].replace(',', ''))
                if jurisdiction not in self.df.jurisdiction.values:
                    self.df.loc[len(self.df)] = [jurisdiction, distribution, initiate_1st, distribution_rate, initiate_1st_rate, 
                                                 np.nan, np.nan]
                    print('[get_df] {0} jurisdiction: {1}, distribution: {2}, initiate_1st: {3}'.format(len(self.df), jurisdiction, distribution, initiate_1st))
        
        # Hawaii, Michigan, New Jersey, District of Columbia
        self.df.loc[len(self.df)] = ['Hawaii', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        self.df.loc[len(self.df)] = ['Michigan', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        self.df.loc[len(self.df)] = ['New Jersey', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        self.df.loc[len(self.df)] = ['District of Columbia', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        self.df = self.df.drop_duplicates(subset=['jurisdiction'])
        self.df = self.df.sort_values(by = 'jurisdiction')
        self.df = self.df.reset_index(drop = True)
        
    def get_df_territory(self):
        ################ Territories ################
        element = self.driver.find_element_by_xpath('//*[@id="vaccinations-territories-list"]/ul')
        elements = element.find_elements_by_tag_name("li")
        for ele in elements:
            # '{"Date":"2021-01-02","MMWR_week":53,"Location":"GU","ShortName":"GUA","LongName":"Guam",
            # "Doses_Distributed":19300,"Doses_Administered":3791,"Dist_Per_100K":11643,"Admin_Per_100K":2287,"
            # Census2019":165768,"dense_rank":50,"ntile":6,"bin":1,"class":1,"avg":2507.5714285714284,"pavg":2507.5714285714284,"navg":1003.5}'
            dictionary = json.loads(ele.get_attribute('data-paired'))
            jurisdiction = dictionary['LongName']
            distribution = dictionary['Doses_Distributed']
            distribution_rate = dictionary['Dist_Per_100K']
            initiate_1st = dictionary['Doses_Administered']
            initiate_1st_rate = dictionary['Admin_Per_100K']
            print('[get_df] jurisdiction: {0}, distribution: {1}, initiate_1st: {2}'.format(jurisdiction, distribution, initiate_1st))
            self.df.loc[len(self.df)] = [jurisdiction, distribution, initiate_1st, distribution_rate, initiate_1st_rate, 
                                         np.nan, np.nan]
    
        ################ Territories ################
        
    def get_df_entity(self):
        ################ Federal Entities ################
        element = self.driver.find_element_by_xpath('//*[@id="vaccinations-federal-list"]/ul')
        elements = element.find_elements_by_tag_name("li")
        for ele in elements:
            # '{"Date":"2021-01-02","MMWR_week":53,"Location":"VA2","ShortName":"VA2","LongName":"Veterans Health",
            # "Doses_Distributed":465425,"Doses_Administered":116374,"Dist_Per_100K":0,"Admin_Per_100K":0,
            # "Census2019":0,"dense_rank":1,"ntile":1,"bin":6,"class":4,"avg":559.3333333333334,"pavg":1079.6666666666667,"navg":1891.9}'
            dictionary = json.loads(ele.get_attribute('data-paired'))
            jurisdiction = dictionary['LongName']
            distribution = dictionary['Doses_Distributed']
            distribution_rate = dictionary['Dist_Per_100K']
            initiate_1st = dictionary['Doses_Administered']
            initiate_1st_rate = dictionary['Admin_Per_100K']
            print('[get_df] jurisdiction: {0}, distribution: {1}, initiate_1st: {2}'.format(jurisdiction, distribution, initiate_1st))
            self.df.loc[len(self.df)] = [jurisdiction, distribution, initiate_1st, distribution_rate, initiate_1st_rate, 
                                         np.nan, np.nan]
        ################ Federal Entities ################

        
        print('[get_df] completed')


    def parse(self):
        '''
        Pipeline
        '''
        
        self.start_driver()
        self.get_page(self.url)

        self.get_headers()
        self.get_update_time()
        self.get_df_state()
        
        self.get_df_territory()
        self.get_df_entity()
        self.get_df_total()
        
        self.close_driver()

In [ ]:
self = CDCSpider()

In [ ]:
self.parse()

In [ ]:
# Hawaii, Michigan, New Jersey, District of Columbia
pd.set_option('display.max_rows', self.df.shape[0]+1)
self.df

In [ ]:
# # Hawaii, Michigan, New Jersey, District of Columbia 
# the data of which are not accessible by the spider
# need to be added manually
data_added = {'Hawaii': [154150, 36605, 10887, 2585], 
              'Michigan': [772150, 291519, 7732, 2919], 
              'New Jersey': [654900, 262982, 7373, 2961], 
              'District of Columbia': [68325, 33049, 9681, 4683]}
for i, state in enumerate(['Hawaii', 'Michigan', 'New Jersey', 'District of Columbia']):
    print(round(data_added[state][0]*data_added[state][3]/(data_added[state][1]*data_added[state][2]), 4))
    self.df.at[self.df.loc[self.df['jurisdiction'] == state].index[0], 'distribution'] = data_added[state][0]
    self.df.at[self.df.loc[self.df['jurisdiction'] == state].index[0], 'initiate_1st'] = data_added[state][1]
    self.df.at[self.df.loc[self.df['jurisdiction'] == state].index[0], 'distribution_rate'] = data_added[state][2]
    self.df.at[self.df.loc[self.df['jurisdiction'] == state].index[0], 'initiate_1st_rate'] = data_added[state][3]

In [ ]:
self.df.to_csv(_DATA_CDC_PATH_ + 'vaccine_{0}.csv'.format(self.update_time_str), index = False)

In [ ]:
# 2021-01-05
data_added = {'Hawaii': [82500, 20893, 5827, 1476], 
              'Michigan': [478800, 99040, 4794, 992], 
              'New Jersey': [390900, 120947, 4401, 1362], 
              'District of Columbia': [35475, 15853, 5027, 2246]}

# 2021-01-06
data_added = {'Hawaii': [82500, 22896, 5827, 1617], 
              'Michigan': [480750, 124689, 4814, 1249], 
              'New Jersey': [406500, 137586, 4577, 1549], 
              'District of Columbia': [35475, 17475, 5027, 2476]}

# 2021-01-07
data_added = {'Hawaii': [95200, 24558, 6724, 1734], 
              'Michigan': [662450, 137887, 6633, 1381], 
              'New Jersey': [572250, 155458, 6443, 1750], 
              'District of Columbia': [45950, 19366, 6511, 2744]}

# 2021-01-08
data_added = {'Hawaii': [95700, 27375, 6759, 1933], 
              'Michigan': [662550, 156251, 6634, 1565], 
              'New Jersey': [572250, 156021, 6443, 1757], 
              'District of Columbia': [49250, 21681, 6978, 3072]}

# 2021-01-11
data_added = {'Hawaii': [115250, 31462, 8140, 2222], 
              'Michigan': [765900, 222379, 7669, 2227], 
              'New Jersey': [651000, 220828, 7329, 2486], 
              'District of Columbia': [60775, 29228, 8611, 4141]}

# 2021-01-12 Updated: Dec 20 2001 As of 9:00am ET
data_added = {'Hawaii': [141750, 33483, 10011, 2365], 
              'Michigan': [768150, 227685, 7692, 2280], 
              'New Jersey': [654900, 221045, 7373, 2489], 
              'District of Columbia': [62725, 29812, 8888, 4224]}

# 2021-01-13
data_added = {'Hawaii': [154150, 36605, 10887, 2585], 
              'Michigan': [772150, 291519, 7732, 2919], 
              'New Jersey': [654900, 262982, 7373, 2961], 
              'District of Columbia': [68325, 33049, 9681, 4683]}